In [4]:
import re
import pandas as pd

# Exemple de DataFrame avec des descriptions d'offres d'emploi
data = {'description': [
    "Nous recherchons un chef de projet (h/f) pour notre entreprise.",
    "Engineer (m/f/d) required for international project.",
    "Verkäufer (m/w) für unser Team gesucht.",
    "Looking for a Senior Developer (h/f) with 5 years of experience.",
    "We need a Data Scientist h/f in our Paris office."
]}
df = pd.DataFrame(data)

# Liste des abréviations à détecter, incluant les parenthèses
abbreviations = r"\(?h/f\)?|\(?m/f\)?|\(?m/w\)?|\(?m/v\)?|\(?m/k\)?|\(?m/n\)?|\(?m/ž\)?|\(?f/n\)?|\(?b/f\)?|\(?άν/γυν\)?|\(?м/ж\)?"

# Fonction pour extraire les 5 mots précédant l'abréviation
def extract_job_title(line):
    # Regex pour capturer les 5 mots précédant l'abréviation
    match = re.search(r'(\b\w+\b[\s,]*){1,5}(?=\s*(' + abbreviations + '))', line, re.IGNORECASE)
    if match:
        return match.group().strip()
    return None

# Appliquer la fonction à la colonne 'description' pour extraire les libellés de poste
df['job_title'] = df['description'].apply(extract_job_title)

# Afficher le DataFrame avec les résultats
print(df[['description', 'job_title']])


                                         description  \
0  Nous recherchons un chef de projet (h/f) pour ...   
1  Engineer (m/f/d) required for international pr...   
2            Verkäufer (m/w) für unser Team gesucht.   
3  Looking for a Senior Developer (h/f) with 5 ye...   
4  We need a Data Scientist h/f in our Paris office.   

                        job_title  
0   recherchons un chef de projet  
1                        Engineer  
2                       Verkäufer  
3  Looking for a Senior Developer  
4        We need a Data Scientist  


In [11]:
import re

import s3fs
import os

def get_file_system() -> s3fs.S3FileSystem:
    """
    Return the s3 file system.
    """
    return s3fs.S3FileSystem(
        client_kwargs={"endpoint_url": f"https://{os.environ['AWS_S3_ENDPOINT']}"},
        key="59GW2Q1OXSLD5ER6S44I",
        secret="P7Dqk6KODsEQO+PBAVlus2sgiY+eJ0DSDDmAIlmE",
        token="eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiI1OUdXMlExT1hTTEQ1RVI2UzQ0SSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSJdLCJhdXRoX3RpbWUiOjE3MjY2NzQyNjMsImF6cCI6Im9ueXhpYSIsImVtYWlsIjoieXZlcy1sYXVyZW50LmJlbmljaG91QGluc2VlLmZyIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImV4cCI6MTcyNzcwMTU5NSwiZmFtaWx5X25hbWUiOiJCZW5pY2hvdSIsImdpdmVuX25hbWUiOiJZdmVzLUxhdXJlbnQiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJhZGRvayIsImFkcmVzc2VzLXJldSIsImFwZSIsImNvZGlmaWNhdGlvbi1wY3MiLCJkZWR1cC1vamEiLCJlc2Etbm93Y2FzdGluZyIsImZ1bmF0aG9uIiwiaGFja2F0aG9uLW50dHMtMjAyMyIsInByZWRpY2F0IiwicmVsZXZhbmMiLCJzaWduZXMtZGUtdmllIiwic2l0ZXNzcGxhYiIsInNzcGxhYiJdLCJpYXQiOjE3MjcwOTY3OTQsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiJlNWYxMjkxYy04NjZhLTQ2NmItYjFlZi05ZmVmNmJkNWNiYjYiLCJsb2NhbGUiOiJlbiIsIm5hbWUiOiJZdmVzLUxhdXJlbnQgQmVuaWNob3UiLCJwb2xpY3kiOiJzdHNvbmx5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZG9oYW9xIiwicm9sZSI6InJvbGUxIiwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzaWQiOiJmOGY5MzE5MC1kODc0LTRmMmQtODY3NC02OTkxZmQ5NWI5NDYiLCJzdWIiOiIxYmFlYjY3MC01YmI4LTRhZDEtOGQzMS1lNjM5ZDllYmRjZGIiLCJ0eXAiOiJCZWFyZXIifQ.QZzwcIyB7fScKsvrZ72JlOCxWY7cp4cBfqJPtLF-rWzUHjw5pKya7TLArV9B91ntWr_x6YxlyAeGNUrMtZeDyw"
    )



import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from langchain_text_splitters import RecursiveCharacterTextSplitter

from src.constants.paths import URL_DATASET, URL_DATASET_WITH_LANG
from src.detect_lang.detect import detect_language, process_data_lang_detec
#from src.utils.data import get_file_system
from src.utils.mapping import id_881693105_desc

DESC_CUTOFF_SIZE = 500

fs = get_file_system()
eol_regex = re.compile(r"\r|\n")

with fs.open(URL_DATASET) as f:
    data = pd.read_csv(f, dtype=str)

# Manually set specific description for one ID
data.loc[data["id"] == "881693105", "description"] = id_881693105_desc

# Process the dataset with cleaning and job title extraction
data = process_data_lang_detec(data)

# Detect language and score
data[["lang", "score"]] = (
    data["description_clean"]
    .str.replace(eol_regex, " ", regex=True)
    .apply(detect_language)
    .apply(pd.Series)
)

# Truncate the description
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=DESC_CUTOFF_SIZE,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
    separators=["\n\n", "\n", ".", "?", "!", ";", ":", ",", " ", ""],
)

for idx, row in data.iterrows():
    splitted_text = text_splitter.split_text(row.description_clean)
    text_truncated = ""
    i = 0
    while (len(text_truncated) < DESC_CUTOFF_SIZE) and (i < len(splitted_text)):
        text_truncated += f" {splitted_text[i]}"
        i += 1
    data.loc[idx, "description_truncated"] = text_truncated
"""
pq.write_to_dataset(
    pa.Table.from_pandas(data),
    root_path=URL_DATASET_WITH_LANG,
    partition_cols=["lang"],
    basename_template="part-{i}.parquet",
    existing_data_behavior="overwrite_or_ignore",
    filesystem=fs,
)
"""

# # Fonction pour extraire les 5 mots précédant l'abréviation
# def extract_job_title(line):
#     # Regex pour capturer les 5 mots précédant l'abréviation
#     match = re.search(r"(\b\w+\b[\s,]*){1,5}(?=\s*(" + abbreviations + "))", line, re.IGNORECASE)
#     if match:
#         return match.group().strip()
#     return None

# # Appliquer la fonction extract_job_title aux colonne 'description' et 'title' pour extraire les libellés de poste
# data["description_job_title"] = data["description"].apply(extract_job_title)
# data["title_job_title"] = data["title"].apply(extract_job_title)



'\npq.write_to_dataset(\n    pa.Table.from_pandas(data),\n    root_path=URL_DATASET_WITH_LANG,\n    partition_cols=["lang"],\n    basename_template="part-{i}.parquet",\n    existing_data_behavior="overwrite_or_ignore",\n    filesystem=fs,\n)\n'

In [4]:
print(data['lang'].unique())

['en' 'sv' 'bg' 'es' 'sk' 'lv' 'sl' 'it' 'nl' 'lt' 'cs' 'et' 'hu' 'pl'
 'el' 'de' 'ro' 'fr' 'pt' 'hr' 'fi' 'ru' 'da' 'sh' 'ca' 'ja' 'gl' 'sr']


In [5]:
print(data[['score','lang']][data['score']<0.4])

          score lang
128    0.256520   sl
133    0.306960   en
224    0.291793   nl
443    0.359631   hr
475    0.285939   sl
...         ...  ...
25207  0.325047   ro
25352  0.358309   en
25356  0.255807   en
25409  0.171804   en
25556  0.218276   sl

[289 rows x 2 columns]


In [12]:
data['lang'] = data.apply(lambda row: 'un' if row['score'] < 0.4 else row['lang'], axis=1)

In [15]:
print(data['lang'][data['lang']=='un'])

128      un
133      un
224      un
275      un
443      un
         ..
25207    un
25352    un
25356    un
25409    un
25556    un
Name: lang, Length: 313, dtype: object


In [7]:
from src.utils.mapping import lang_mapping
#country_map = lang_mapping.loc[lang_mapping["lang_iso_2"].isin(list_country)]

In [8]:
print(lang_mapping["lang_iso_2"])

0     en
1     de
2     bg
3     hr
4     da
5     es
6     et
7     fi
8     fr
9     el
10    hu
11    ga
12    it
13    lv
14    lt
15    mt
16    nl
17    pl
18    pt
19    ro
20    sk
21    sl
22    sv
23    cs
24    un
Name: lang_iso_2, dtype: object


In [14]:
from src.utils.mapping import lang_mapping
data['lang'] = data['lang'].where(data['lang'].isin(lang_mapping["lang_iso_2"]), 'un')

In [9]:
print(data['lang'].isin(lang_mapping["lang_iso_2"]))

0        True
1        True
2        True
3        True
4        True
         ... 
25660    True
25661    True
25662    True
25663    True
25664    True
Name: lang, Length: 25665, dtype: bool
